In [8]:
from OpenAI_LLM import OpenAI_LLM

In [9]:
llm = OpenAI_LLM()

In [10]:
llm('nihao')

'你好！有什么我可以帮助你的吗？'

In [26]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

schema = Object(
    id="script",
    description="Adapted from the novel into script",
    attributes=[
        Text(
            id="role",
            description="The character who is speaking or performing an action",
        ),
        Text(
            id="dialogue",
            description="The dialogue spoken by the characters in the sentence",
        ),
        Text(
            id="action",
            description='''The actions performed by the characters in the text, A high-level summary of a character's behavior. action equals "对话" or "独白" if it's dialogue in sentence, equals other action if it's no dialogue''',
        )
    ],
    examples=[
        (
            '''德妃拈了一枚吹花红宝钿在手中把玩，轻笑道：“难为皇上也没生气，只安慰了瑛贵嫔几句。”我淡淡一笑，拿着一枝玉搔头拨着耳垂，“咱们的皇上是什么性子，生气也未必即刻说出来，何况又是平日里最喜欢的表妹。”''',
            [
                {"role": "德妃", "dialogue": "难为皇上也没生气，只安慰了瑛贵嫔几句。","action":"轻笑"},
                {"role": "我", "dialogue": "咱们的皇上是什么性子，生气也未必即刻说出来，何况又是平日里最喜欢的表妹。","action":"淡淡一笑"},
            ],
        ),
        (
            '''后宫诸人本就眼红恬嫔的身孕，如此一来更是嫉妒，谨慎如悫妃也颇有微词：“才四个月怎能知道是男是女，臣妾怀皇长子时到六月间太医断出是男胎，皇上也只是按礼制在臣妾初有喜脉时加以封赏晋为贵嫔，并未有其他破例。”而皇后伸手拈了一枚樱桃吃了，方慢慢道：“恬嫔几次三番说有胎动不安的症状，皇上也只是为了安抚她才这样做。为皇家子嗣计，本宫是不会有异议的。”''',
            [
                {"role": "悫妃", "dialogue": "才四个月怎能知道是男是女，臣妾怀皇长子时到六月间太医断出是男胎，皇上也只是按礼制在臣妾初有喜脉时加以封赏晋为贵嫔，并未有其他破例。","action":"对话"},
                {"role": "皇后", "dialogue": "恬嫔几次三番说有胎动不安的症状，皇上也只是为了安抚她才这样做。为皇家子嗣计，本宫是不会有异议的。","action":"对话"},
            ],
        )
    ],
    many=True,
)

In [15]:
chain = create_extraction_chain(llm, schema)
print(chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

script: Array<{ // Adapted from the novel into script
 role: string // The character who is speaking or performing an action
 dialogue: string // The dialogue spoken by the characters in the sentence
 action: string // The actions performed by the characters in the text, A high-level summary of a character's behavior. action equals "对话" or "独白" if it's dialogue in sentence, equals other action if it's no dialogue
}>
```


Please output the extracted information in CSV format in Excel dialect. Please use a | as the delimiter. 
 Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.



Input: 我还记得跟丈夫提出离婚时他那一脸的惊愕表情：“你以为到了你这个

In [22]:
text = '''
白衬衣上总是沾着一块无法洗去的油污，头发蓬乱，神色显得有点低迷。他那副模样根本不像个音乐老师，倒像是偶然跑到校园里来的一个流浪艺术家。他比我大十五岁，还是两个孩子的父亲。按说我无论如何都不会爱上这样一个人，他却硬生生地闯进我的梦里。一个女孩喜欢一个男人时总是不管不顾。我跟他说我总是想一个人，想得什么也干不下去，他有点蒙：“是谁？”我说：“是你。”说这话是在一个周末的傍晚，我在他的宿舍门口堵住了他。他抬手挠了挠头发，像不认识似的上下打量了我一眼，嘴角微微朝左侧一歪：“这可不行。”
'''

In [23]:
response= chain.run(text)

In [24]:
if 'script' in response['data']:
    for item in response['data']['script']:
        print(item)

{'role': '音乐老师', 'dialogue': '白衬衣上总是沾着一块无法洗去的油污，头发蓬乱，神色显得有点低迷。他那副模样根本不像个音乐老师，倒像是偶然跑到校园里来的一个流浪艺术家。', 'action': '无'}
{'role': '我', 'dialogue': '他比我大十五岁，还是两个孩子的父亲。按说我无论如何都不会爱上这样一个人，他却硬生生地闯进我的梦里。一个女孩喜欢一个男人时总是不管不顾。', 'action': '无'}
{'role': '我', 'dialogue': '我跟他说我总是想一个人，想得什么也干不下去，他有点蒙：“是谁？”我说：“是你。”说这话是在一个周末的傍晚，我在他的宿舍门口堵住了他。', 'action': '无'}
{'role': '音乐老师', 'dialogue': '他抬手挠了挠头发，像不认识似的上下打量了我一眼，嘴角微微朝左侧一歪：“这可不行。”', 'action': '无'}


In [25]:
response

{'data': {'script': [{'role': '音乐老师',
    'dialogue': '白衬衣上总是沾着一块无法洗去的油污，头发蓬乱，神色显得有点低迷。他那副模样根本不像个音乐老师，倒像是偶然跑到校园里来的一个流浪艺术家。',
    'action': '无'},
   {'role': '我',
    'dialogue': '他比我大十五岁，还是两个孩子的父亲。按说我无论如何都不会爱上这样一个人，他却硬生生地闯进我的梦里。一个女孩喜欢一个男人时总是不管不顾。',
    'action': '无'},
   {'role': '我',
    'dialogue': '我跟他说我总是想一个人，想得什么也干不下去，他有点蒙：“是谁？”我说：“是你。”说这话是在一个周末的傍晚，我在他的宿舍门口堵住了他。',
    'action': '无'},
   {'role': '音乐老师',
    'dialogue': '他抬手挠了挠头发，像不认识似的上下打量了我一眼，嘴角微微朝左侧一歪：“这可不行。”',
    'action': '无'}]},
 'raw': 'role|dialogue|action\n音乐老师|白衬衣上总是沾着一块无法洗去的油污，头发蓬乱，神色显得有点低迷。他那副模样根本不像个音乐老师，倒像是偶然跑到校园里来的一个流浪艺术家。|无\n我|他比我大十五岁，还是两个孩子的父亲。按说我无论如何都不会爱上这样一个人，他却硬生生地闯进我的梦里。一个女孩喜欢一个男人时总是不管不顾。|无\n我|我跟他说我总是想一个人，想得什么也干不下去，他有点蒙：“是谁？”我说：“是你。”说这话是在一个周末的傍晚，我在他的宿舍门口堵住了他。|无\n音乐老师|他抬手挠了挠头发，像不认识似的上下打量了我一眼，嘴角微微朝左侧一歪：“这可不行。”|无',
 'errors': [],
 'validated_data': {}}